In [1]:
import ccxt
import pandas as pd
import datetime
import time
import os
import re

# 기본 설정
binance = ccxt.binance(config={
    'options': {"defaultType" : "future"}
})

### 데이터 다운 및 업데이트 하기
def Save_and_Update_Data(coin, save_path):
    file_name = coin.replace("/USDT","_M") # 이름 수정
    
    if os.path.exists(os.path.join(save_path, f"{file_name}.xlsx"))==True: # 파일이 존재할 경우
        df = pd.read_excel(os.path.join(save_path,f"{file_name}.xlsx"),index_col=0)
        past = datetime.datetime.strptime(str(df.iloc[-1].name), f'%Y-%m-%d %H:%M:00')
        now = datetime.datetime.now().strftime(f'%Y-%m-%d %H:%M:00')
        now = datetime.datetime.strptime(now, f'%Y-%m-%d %H:%M:00')
        diff = str(now-past)
        h, m, s = list(map(int, diff.split(":")))
        total_time_diff = (h * 60) + m + s # 분으로 환산
        total_time_diff

        if total_time_diff > 0: # 1분 이상 차이 날 경우
            new_ohlcv = binance.fetch_ohlcv(
                f'{coin}', 
                timeframe='1m',
                params={"startTime":df.index[-1]},
                limit=total_time_diff
            )
            new_df = pd.DataFrame(new_ohlcv, columns=['datetime','open','high','low','close','volume'])
            Cal_DataFrame(new_df) # OHCLV 값 기반으로 계산
            new_df['datetime'] = pd.to_datetime(new_df['datetime'], unit='ms') + datetime.timedelta(hours=9)
            new_df.set_index('datetime',inplace=True)
            df = pd.concat([df,new_df])
#             print(f"[{coin}] Update {total_time_diff} minutes")
        else: # 1분 이하로 초단위로 차이 날 경우, 맨 마지막 행 업데이트
            new_ohlcv = binance.fetch_ohlcv(f'{coin}', timeframe='1m',limit=1)
            new_df = pd.DataFrame(new_ohlcv, columns=['datetime','open','high','low','close','volume'])
            Cal_DataFrame(new_df) # OHCLV 값 기반으로 계산
            new_df['datetime'] = pd.to_datetime(new_df['datetime'], unit='ms') + datetime.timedelta(hours=9)
            new_df.set_index('datetime',inplace=True)
            df = pd.concat([df[:-1],new_df])
#             print(f"[{coin}] Update Under 1 minutes")
    else: # 파일이 존재하지 않을 경우
        ### Data가 없을 때 1500개 1분봉 데이터 다운
#         print(f"[{coin}] New Data Saving . . .")
        ohlcv = binance.fetch_ohlcv(f'{coin}', timeframe='1m',limit=1500)
        df = pd.DataFrame(ohlcv, columns=['datetime','open','high','low','close','volume'])
        Cal_DataFrame(df) # OHCLV 값 기반으로 계산
        df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') + datetime.timedelta(hours=9)
        df.set_index('datetime',inplace=True)
        time.sleep(0.5)

    df = df.loc[~df.index.duplicated(keep='last')] #최신행만 남겨두고 중복된 행 제거
    df.to_excel(f'{os.path.join(save_path, file_name)}.xlsx') # 엑셀 데이터로 저장
    
    return df
    
def Cal_DataFrame(DataFrame): #[time, open, high, low, close, volume]
    df = DataFrame
    
    # 가중치를 둔 거래대금
    df['Cal_Value'] = (df['open']*0.20+df['high']*0.15+df['low']*0.15+df['close']*0.50)*df['volume']
     

    return df

In [2]:
# 메인
markets = [x for x in binance.fetch_tickers().keys() if x.endswith("/USDT")] # 선물 마켓 시장 티커 전부 가져오기
path = os.path.join(os.getcwd(),"Save Data") # 데이터 저장 위치

print("*-------------Start-------------*")
while True:
    for coin in markets: # 코인 한개씩 조회
        df = Save_and_Update_Data(coin,path) # 데이터 저장 및 업데이트
        time.sleep(0.01)
    break
    
print("*--------------End--------------*")

*-------------Start-------------*


ValueError: invalid literal for int() with base 10: '2 days, 18'

In [195]:
len(markets)

142